In [66]:
import os, random, argparse
import numpy as np
import tensorflow.keras.backend as K
from tensorflow.keras.utils import multi_gpu_model
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau, LearningRateScheduler, EarlyStopping, TerminateOnNaN, LambdaCallback
from tensorflow_model_optimization.sparsity import keras as sparsity
from tensorflow.python.keras import optimizers
from yolo3.model import get_yolo3_train_model
from yolo3.data import yolo3_data_generator_wrapper
from yolo2.model import get_yolo2_train_model
from yolo2.data import yolo2_data_generator_wrapper
from common.utils import get_classes, get_anchors, get_dataset, optimize_tf_gpu
from common.model_utils import get_optimizer
#from common.callbacks import EvalCallBack, DatasetShuffleCallBack

In [67]:
os.environ['TF_ENABLE_AUTO_MIXED_PRECISION'] = '1'
os.environ['TF_AUTO_MIXED_PRECISION_GRAPH_REWRITE_IGNORE_PERFORMANCE'] = '1'

import tensorflow as tf
optimize_tf_gpu(tf, K)

In [68]:
#===== data options
model_type='yolo3_mobilenet_lite'
anchors_path='configs/yolo3_anchors.txt'
model_image_size='416x416'
weights_path=None
annotation_file='trainval.txt'
val_annotation_file= None
val_split=0.1
classes_path='configs/voc_classes.txt'

In [69]:
#===== training options
batch_size=16
optimizerMethod='Adam'
learning_rate=1e-3
decay_type= None
transfer_epoch=20
freeze_level= None #0:NA/1:backbone/2:only open prediction layer
init_epoch=0
total_epoch=3 # default = 250
multiscale=False
rescale_interval=10
model_pruning=False
label_smoothing=0
data_shuffle=False
gpu_num=1

In [70]:
#======= eval options
eval_online=False
eval_epoch_interval=10
save_eval_checkpoint=False

In [71]:
height, width = model_image_size.split('x')
model_image_size = (int(height), int(width))

In [52]:
#======= convert anchors
import csv
import numpy as np
#def cvtAnchots(anchors_path):
with open(anchors_path,'r') as rd:
    reader= csv.reader(rd, delimiter= ',')
    xs=[]
    for row in reader:
        xs.append(row)
    xs= np.array(xs, dtype= int)
x1=[]; x2=[]
xt=[]
count=0
for x in (xs[0]):
    count+=1
    if (count % 2 == 1) :
        x1.append(x)
    else:
        x2.append(x)
count2=0;count3=0
for x in x1:
    count2+=1
    for y in x2:
        count3+=1
        if count2== count3:
            xt.append([x,y])
    count3=0
xt=np.array(xt,dtype='float32')
#x=np.array((x1,x2),dtype=int)
#print(x1,x2)
print(xt)
    #return xt

[[ 10.  13.]
 [ 16.  30.]
 [ 33.  23.]
 [ 30.  61.]
 [ 62.  45.]
 [ 59. 119.]
 [116.  90.]
 [156. 198.]
 [373. 326.]]


In [ ]:
#============ Main train 
log_dir = 'logs/000/'
callbacks= None

class_names = get_classes(classes_path)
num_classes = len(class_names)
#print(num_classes)
#anchors = anchors_path
#anchors = cvtAnchots(anchors_path)
anchors = get_anchors(anchors_path)
num_anchors = len(anchors)
#print (anchors)

# get freeze level according to CLI option
if weights_path:
    freeze_level = 0
else:
    freeze_level = 1

if freeze_level is not None:
    freeze_level = freeze_level


# callbacks for training process
logging = TensorBoard(log_dir=log_dir, histogram_freq=0, write_graph=False, write_grads=False, write_images=False, update_freq='batch')
checkpoint = ModelCheckpoint(log_dir + 'ep{epoch:03d}-loss{loss:.3f}-val_loss{val_loss:.3f}.h5',
    monitor='val_loss',
    verbose=1,
    save_weights_only=False,
    save_best_only=True,
    period=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, verbose=1, cooldown=0, min_lr=1e-10)
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=50, verbose=1)
terminate_on_nan = TerminateOnNaN()

#************ remark callbacks
#callbacks=[logging, checkpoint, reduce_lr, early_stopping, terminate_on_nan]

# get train&val dataset
dataset = get_dataset(annotation_file)
if val_annotation_file:
    val_dataset = get_dataset(val_annotation_file)
    num_train = len(dataset)
    num_val = len(val_dataset)
    dataset.extend(val_dataset)
else:
    val_split = val_split
    num_val = int(len(dataset)*val_split)
    num_train = len(dataset) - num_val

# get different model type & train&val data generator
if num_anchors == 9:
    # YOLOv3 use 9 anchors
    get_train_model = get_yolo3_train_model
    data_generator = yolo3_data_generator_wrapper
    tiny_version = False
elif num_anchors == 6:
    # Tiny YOLOv3 use 6 anchors
    get_train_model = get_yolo3_train_model
    data_generator = yolo3_data_generator_wrapper
    tiny_version = True
elif num_anchors == 5:
    # YOLOv2 use 5 anchors
    get_train_model = get_yolo2_train_model
    data_generator = yolo2_data_generator_wrapper
    tiny_version = False
else:
    raise ValueError('Unsupported anchors number')
'''
#*********** remark callbacks
# prepare online evaluation callback
if eval_online:
    eval_callback = EvalCallBack(model_type, dataset[num_train:], anchors, class_names, model_image_size, model_pruning, log_dir, eval_epoch_interval=eval_epoch_interval, save_eval_checkpoint=save_eval_checkpoint)
    callbacks.append(eval_callback)

# prepare train/val data shuffle callback
if data_shuffle:
    shuffle_callback = DatasetShuffleCallBack(dataset)
    callbacks.append(shuffle_callback)

# prepare model pruning config
pruning_end_step = np.ceil(1.0 * num_train / batch_size).astype(np.int32) * total_epoch
if model_pruning:
    pruning_callbacks = [sparsity.UpdatePruningStep(), sparsity.PruningSummaries(log_dir=log_dir, profile_batch=0)]
    callbacks = callbacks + pruning_callbacks
'''
# prepare optimizer
optimizer = get_optimizer(optimizerMethod, learning_rate, decay_type=None)

# get train model
model = get_train_model(model_type, anchors, num_classes, weights_path=weights_path, freeze_level=freeze_level, optimizer=optimizer, label_smoothing=label_smoothing, model_pruning=model_pruning, pruning_end_step=pruning_end_step)
# support multi-gpu training
template_model = None
if gpu_num >= 2:
    # keep the template model for saving result
    template_model = model
    model = multi_gpu_model(model, gpus=gpu_num)
    # recompile multi gpu model
    model.compile(optimizer=optimizer, loss={'yolo_loss': lambda y_true, y_pred: y_pred})
model.summary()

# Transfer training some epochs with frozen layers first if needed, to get a stable loss.
input_shape = model_image_size
#******** remark assert
#assert (input_shape[0]%32 == 0 and input_shape[1]%32 == 0),'Multiples of 32 required'
initial_epoch = init_epoch
epochs = initial_epoch + transfer_epoch
print("Transfer training stage")
print('Train on {} samples, val on {} samples, with batch size {}, input_shape {}.'.format(num_train, num_val, batch_size, input_shape))
print('callbacks=', callbacks)
model.fit_generator(data_generator(dataset[:num_train], batch_size, input_shape, anchors, num_classes),
        steps_per_epoch=max(1, num_train/batch_size),
        validation_data=data_generator(dataset[num_train:], batch_size, input_shape, anchors, num_classes),
        validation_steps=max(1, num_val/batch_size),
        epochs=epochs,
        initial_epoch=initial_epoch,
        workers=1,
        use_multiprocessing=False)#,
        #callbacks=callbacks)

if decay_type:
    # rebuild optimizer to apply learning rate decay, only after
    # unfreeze all layers
    #***************remark callbacks
    #callbacks.remove(reduce_lr)
    steps_per_epoch = max(1, num_train//batch_size)
    decay_steps = steps_per_epoch * (total_epoch - init_epoch - transfer_epoch)
    optimizer = get_optimizer(optimizer, learning_rate, decay_type=decay_type, decay_steps=decay_steps)

# Unfreeze the whole network for further tuning
# NOTE: more GPU memory is required after unfreezing the body
print("Unfreeze and continue training, to fine-tune.")
for i in range(len(model.layers)):
    model.layers[i].trainable = True
model.compile(optimizer=optimizer, loss={'yolo_loss': lambda y_true, y_pred: y_pred}) # recompile to apply the change


if args.multiscale:
    rescale_interval = rescale_interval
else:
    rescale_interval = -1  #Doesn't rescale

print('Train on {} samples, val on {} samples, with batch size {}, input_shape {}.'.format(num_train, num_val, batch_size, input_shape))
model.fit_generator(data_generator(dataset[:num_train], batch_size, input_shape, anchors, num_classes, rescale_interval),
    steps_per_epoch=max(1, num_train//batch_size),
    validation_data=data_generator(dataset[num_train:], batch_size, input_shape, anchors, num_classes),
    validation_steps=max(1, num_val//batch_size),
    epochs=total_epoch,
    initial_epoch=epochs,
    workers=1,
    use_multiprocessing=False,
    callbacks=callbacks)

# Finally store model
if args.model_pruning:
    if template_model is not None:
        template_model = sparsity.strip_pruning(template_model)
    else:
        model = sparsity.strip_pruning(model)

if template_model is not None:
    template_model.save(log_dir + 'trained_final.h5')
else:
    model.save(log_dir + 'trained_final.h5')

C:\Users\user\.conda\envs\tensorflow2\lib\site-packages\keras_applications\mobilenet.py:207: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


Create  YOLOv3 yolo3_mobilenet_lite model with 9 anchors and 20 classes.
model layer number: 181
Freeze the first 87 layers of total 181 layers.
Model: "model_29"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
image_input (InputLayer)        [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           image_input[0][0]                
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 3 864         conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, None, 

 10/931 [..............................] - ETA: 3:22:31 - loss: 5242.0725 - location_loss: 10.1055 - confidence_loss: 5208.5967 - class_loss: 21.0892